In [1]:
# imports
import os
import yaml
import numpy as np
import pandas as pd
import theano
import lasagne
import loading
from training import *
from network import *
import architectures as arches

# aliases
L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor

# directories
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir_ = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/')
datadir = os.path.join(headdir, 'Data/model input')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nnets')

# loading data
data = loading.default_loader(os.path.join(datadir, '1-4 (no computer).csv'))
hvhdata = loading.default_loader(os.path.join(datadir, '0 (with groups).csv'))
Xs = np.concatenate(hvhdata[2])
ys = np.concatenate(hvhdata[3])
Ss = np.concatenate(hvhdata[4])

# Prototyping

In [ ]:
with open('arch_specs.yaml') as archfile:
    arch_dict = yaml.load(archfile)

for name in ['g1', 'g2', 'g4', 'h1', 'h2', 'h4']:
    paramsdir = os.path.join(paramsdir_, name[:-1])
    architecture = arch_dict[name]

    af = getattr(arches, architecture['type'])
    net = af(**architecture['kwargs'])
    print('N params:', L.count_params(net))
    print(architecture)

    run_full_fit(architecture, data, hvhdata, paramsdir=paramsdir, tune=True) 

N params: 868
{'name': 'g1', 'type': 'multiconvX_ws', 'kwargs': {'subnet_specs': [[1, [4, 5]], [1, [4, 6]], [1, [4, 7]], [1, [4, 8]]]}}

Split Number 0
(12362, 2, 4, 9)
Epoch 0 took 2.985s
	training loss:			2.9405
	validation loss:		2.8215
	validation accuracy:		15.19%
	total time elapsed:		3.092s
Epoch 25 took 3.006s
	training loss:			2.6585
	validation loss:		2.6087
	validation accuracy:		19.86%
	total time elapsed:		86.274s
Epoch 50 took 3.162s
	training loss:			2.6418
	validation loss:		2.5822
	validation accuracy:		20.66%
	total time elapsed:		165.124s
Epoch 75 took 3.065s
	training loss:			2.6404
	validation loss:		2.5836
	validation accuracy:		20.60%
	total time elapsed:		248.304s
Epoch 100 took 3.178s
	training loss:			2.6366
	validation loss:		2.5793
	validation accuracy:		20.90%
	total time elapsed:		327.288s
Epoch 125 took 3.031s
	training loss:			2.6367
	validation loss:		2.5750
	validation accuracy:		20.70%
	total time elapsed:		406.451s
Abandon ship!

TEST PERFORMANCE
	St

## Subject tuning

Usually doesn't work (not enough data even for very simple classifier layers)

In [ ]:
dafiname = os.path.join(datadir, '0 (with groups).csv')
subject_data = [loading.default_loader(dafiname, subject=s) for s in range(40)]
arch = archs[archname]

In [ ]:
print([len(s[0]) for s in subject_data])

In [ ]:
for i in range(5):
    pafiname = '{} {} split agg fit exp 1-4.npz'.format(archname, i)
    prenet = Network(arch)
    prenet.load_params(os.path.join(paramsdir, pafiname))
    params = L.get_all_param_values(prenet.net)
    print('PREFIT {}\n'.format(i))
    
    for s in range(40):
        sdata = subject_data[s]
        num_obs = len(sdata[0])
        bs = num_obs//5
#         if num_obs > 50:
        tuner = FineTuner(stopthresh=10, batchsize=bs)
        print('SUBJECT {}\n'.format(s))
        
        for j in range(5):
            fname = '{} {} agg fit exp 1-4 {} subject {} tune fit exp 0'.format(archname, i, s, j)
            net = tuner.train_all(architecture=arch, data=sdata, split=j, startparams=params, freeze=True)
            net.save_params(os.path.join(paramsdir, fname))

## Data aggregation

doesn't need run more than once

In [ ]:
datafilenames = ['0 (with groups)', '1 (with computer)', '2 (with computer)', '3 (with computer)', '4']
datafilenames = [os.path.join(datadir, fname + '.csv') for fname in datafilenames]
colnames = ['subject', 'color', 'bp', 'wp', 'zet', 'rt']

e0 = pd.read_csv(datafilenames[0], names=colnames+['splitno'])
e1 = pd.read_csv(datafilenames[1], names=colnames)
e2 = pd.read_csv(datafilenames[2], names=colnames)
e3 = pd.read_csv(datafilenames[3], names=colnames+['task', 'taskorder', 'session'])
e4 = pd.read_csv(datafilenames[4], names=colnames+['timecondition'])
Es = [e1, e2, e3, e4]
for i, e in enumerate(Es[1:]):
    e['subject'] = e['subject'] + Es[i-1].loc[Es[i-1]['subject']<1000, 'subject'].max()

A = pd.concat([e[colnames] for e in [e1, e2, e3, e4]])

groups = np.arange(len(A))%5 + 1
np.random.seed(100001)
np.random.shuffle(groups)
A['group'] = groups

A.to_csv(os.path.join(datadir, '1-4.csv'), encoding='ASCII', header=False, index=False)
A.loc[A['subject']<1000, :].to_csv(
    os.path.join(datadir, '1-4 (no computer).csv'), 
    encoding='ASCII', header=False, index=False
)